In [1]:
#Gerekli kütüphaneler

from keras.models import load_model
import cv2
import numpy as np
import tkinter
from tkinter import messagebox
import smtplib


In [2]:

# Tkinter'ı Başlat
root = tkinter.Tk()
root.withdraw()

#Colab'ta yaptığımız modeli yükleyelim
model = load_model('Face_Mask_DetectionNew.h5')

#Haarcascademizide ayarlayalım.
face_det_classifier=cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

#Videomuzuda yakalayalım. 
vid_source=cv2.VideoCapture(0)

"""" Maske takma ayrıntılarını ve yüzün etrafındaki dikdörtgenin rengini içeren sözlükler. 
Maske takılıyorsa renk yeşil olur ve maske takılmazsa yüzün etrafındaki dikdörtgen rengi kırmızı olur."""
text_dict={0:'Mask ON',1:'No Mask'}
rect_color_dict={0:(0,255,0),1:(0,0,255)}

SUBJECT = "Subject"   
TEXT = "3.katta maske takmayan birisi var. Lütfen uyarın."
 

# Kamera döngümüz
while(True):

    ret, img = vid_source.read()
    grayscale_img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces = face_det_classifier.detectMultiScale(grayscale_img,1.3,5)  

    for (x,y,w,h) in faces:
    
        face_img = grayscale_img[y:y+w,x:x+w]
        resized_img = cv2.resize(face_img,(112,112))
        normalized_img = resized_img/255.0
        reshaped_img = np.reshape(normalized_img,(1,112,112,1))
        result=model.predict(reshaped_img)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),rect_color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),rect_color_dict[label],-1)
        cv2.putText(img, text_dict[label], (x, y-10),cv2.FONT_HERSHEY_SIMPLEX,0.8,(0,0,0),2) 
        
        # Eğer label 1 ise lütfen maske tak anlamına geliyor ! 0 ise tam tersi!
        if (label == 1):
            messagebox.showwarning("Tehlike","Neden Maske Takmıyorsunuz ??? Lütfen Takın !")
            
            # Maske takmayınca mail gönderme 
            message = 'Subject: {}\n\n{}'.format(SUBJECT, TEXT)
            mail = smtplib.SMTP('smtp.gmail.com', 587)
            mail.ehlo()
            mail.starttls()
            mail.login('zekiunyildiz@gmail.com','*************')
            mail.sendmail('zekiunyildiz@gmail.com','gulcankaraagac5@gmail.com',message)
            mail.close
        else:
            pass
            break

    cv2.imshow('LIVE Video Feed',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

SMTPAuthenticationError: (535, b'5.7.8 Username and Password not accepted. Learn more at\n5.7.8  https://support.google.com/mail/?p=BadCredentials v16sm2657453eds.64 - gsmtp')